In [10]:
%pip install gdown
import gdown

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
url="https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view"

In [8]:
file_id=url.split("/")[-2]
file_id

'1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3'

In [9]:

prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "kidney-CT-Scan-data.zip")

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=21b41180-671f-4f43-b222-f06ecae4192b
To: d:\Data Science\END to END Proj\KidneyDiseaseMLOPS\research\kidney-CT-Scan-data.zip
100%|██████████| 57.7M/57.7M [00:24<00:00, 2.33MB/s]


'kidney-CT-Scan-data.zip'

In [11]:
import os
%pwd

'd:\\Data Science\\END to END Proj\\KidneyDiseaseMLOPS\\research'

In [12]:
os.chdir("../")

In [21]:
## ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [22]:
from src.KidneyClassifier.constant import *
from src.KidneyClassifier.utils.common import read_yaml,create_directories 

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [24]:
import os
import urllib.request as request
import zipfile
import pandas as pd
from KidneyClassifier import logger
from KidneyClassifier.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
     
     '''
        Fetch data from the URL if not already downloaded
        '''
     try:
            zip_download_dir = self.config.local_data_file

            if os.path.exists(zip_download_dir):
                logger.info(f"File already exists at {zip_download_dir}. Skipping download.")
                return

            dataset_url = self.config.source_URL
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

     except Exception as e:
            raise e

        
    

    def extract_zip_file(self):
      """
      Extracts the zip file into the data directory if not already extracted.
      """
      unzip_path = self.config.unzip_dir

    # Check if unzip_path exists and is not empty
      if os.path.exists(unzip_path) and os.listdir(unzip_path):
        logger.info(f"Data already extracted at {unzip_path}. Skipping extraction.")
        return

      os.makedirs(unzip_path, exist_ok=True)
      logger.info(f"Extracting zip file {self.config.local_data_file} to {unzip_path}")

      with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
         zip_ref.extractall(unzip_path)

      logger.info(f"Extraction complete.")


In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-18 21:42:28,519: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-18 21:42:28,522: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-18 21:42:28,530: INFO: common: created directory at: artifacts]
[2025-07-18 21:42:28,532: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-18 21:42:28,535: INFO: 1579108944: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=ce7ed042-5a99-4b98-a21a-92f57adfe8fa
To: d:\Data Science\END to END Proj\KidneyDiseaseMLOPS\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [01:00<00:00, 959kB/s] 

[2025-07-18 21:43:32,597: INFO: 1579108944: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
